In [1]:
import pandas as pd
from transformers import pipeline
import os 
import numpy as np

In [2]:
# read data
data_path = os.path.join('..', 'data', "Louise-Prep-Data-SaraOgMonopolet.json")
df = pd.read_json(data_path)

In [3]:
# log into huggingface:

from dotenv import load_dotenv
import os

load_dotenv()  # reads .env
hf_token = os.getenv("HF_TOKEN")

In [4]:
from huggingface_hub import login
login(new_session=False)

TypeError: login() got an unexpected keyword argument 'new_session'

In [8]:
df['acceptable_solutions'].iloc[3]

['Sæt campingvognen i haven og brug den som klinik/kontor/meditationsrum (og senere evt. gæsteværelse) — det er helt fint, det kan fungere som din virksomhedslokation og kan blive en styrke for dit brand.',
 'Fokuser på kvaliteten af din rådgivning og start med at få kunder fra dit nærmeste netværk (venner og bekendte), så positive oplevelser spreder sig mund-til-mund uafhængigt af, at du sidder i en campingvogn.',
 'Gør campingvognen pæn og gennemtænkt: vedligehold den, overvej at bygge en lille terrasse foran og sørg for, at den præsenterer sig æstetisk godt, så den ikke fremstår slidt.',
 'Brug campingvognen som et anderledes branding-element — fremhæv det som et unikt, mindeværdigt valg (vælg den fremfor et kedeligt kontor i byen) for at tiltrække kunder.',
 'Hvis nogen reagerer negativt på campingvognen, forklar at den er midlertidig (sig fx at den kun står der i en periode) som en mulig forklaring eller afværgning.']

### Gemma-3B

In [15]:
# specify system and user prompts

def set_prompts(dilemma):
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet og du skal gøre dette med maks 50 tokens, men du skal ikke bare give mig et enkelt ord som svar."
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"Her er dilemmaet: {dilemma}"
                }
            ],
        },
    ]
    return messages

In [5]:
def set_prompts(system_prompt, dilemma):
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_prompt
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"Her er dilemmaet: {dilemma}. Råd: "
                }
            ],
        },
    ]
    return messages

In [18]:
system_prompt = "Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet og du skal gøre dette med maks 50 tokens, men du skal ikke bare give mig et enkelt ord som svar."
new_prompt = "Du er deltager i et panel, der diskuterer personlige dilemmaer. Giv mig ikke andet end svaret på dilemmaet. Du må ikke kun give et enkelt ord som svar. Det er meget vigtigt, at du kun bruger maks 50 tokens på svaret."

In [7]:
llama_prompt = "Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. Det er MEGET vigtigt, at du kun skal gøre dette med maks 50 tokens. Du skal heller ikke bare give mig et enkelt ord som svar. Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet. Du skal ikke adressere personen, rådet er til."

In [13]:
optimized_prompt = """Rolle: Du er en ekspert i et panel, der løser personlige dilemmaer med præcision.

Opgave: Giv et konkret og utvetydigt råd til det præsenterede dilemma.

STRIKSE RETNINGSLINJER:
- Format: Gå direkte til rådet uden indledning (undgå f.eks. "Mit råd er" eller "Du bør").
- Stil: Brug neutral, upersonlig stil. Brug ALDRIG ordet "jeg" eller "du" eller 'dig'.
- Længde: MEGET vigtigt det ikke overskrider maks. 80 tokens.
- Forbud: Svaret må ikke bestå af kun ét ord.
"""

In [8]:
# loading model from huggingface Hub
from transformers import AutoProcessor, AutoModelForMultimodalLM

processor = AutoProcessor.from_pretrained("google/gemma-3-12b-it")
model = AutoModelForMultimodalLM.from_pretrained("google/gemma-3-12b-it")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading weights:   0%|          | 0/1065 [00:00<?, ?it/s]

In [12]:
messages = set_prompts(optimized_prompt, df['first_person_dilemma'].iloc[10])
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

print(messages)

[{'role': 'system', 'content': [{'type': 'text', 'text': 'Rolle: Du er en ekspert i et panel, der løser personlige dilemmaer med præcision.\n\nOpgave: Giv et konkret og utvetydigt råd til det præsenterede dilemma.\n\nStrikte retningslinjer:\n- Format: Gå direkte til rådet uden indledning (undgå f.eks. "Mit råd er" eller "Du bør").\n- Stil: Skriv i neutral eller bydende form. Brug aldrig jeg-person eller direkte henvendelse (som "du").\n- Længde: Svaret skal være kortfattet og præcist (maks. 50 tokens).\n- Forbud: Svaret må ikke bestå af kun ét ord.\n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Her er dilemmaet: På min arbejdsplads har min chef Sanne indført "Sjov Hattefredag". Det går simpelt ud på, at alle hver fredag skal møde op iført en sjov hat — det kan være en sombrero, en 12-mands-hat, en badhætte med propeller eller lignende. Min chef har formentlig været på et lederkursus og mener, at det styrker kreativiteten og fællesskabet og giver mulighed for at grine og 

In [13]:
outputs = model.generate(**inputs, max_new_tokens=60)
print(processor.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Forklar over for Sanne, at initiativet skaber ubehag, og at det ikke fremmer arbejdsmiljøet for alle. Anmod om, at det ikke er obligatorisk at deltage.<end_of_turn>


### Test Mistral model

In [62]:
from vllm import LLM
from vllm.sampling_params import SamplingParams
from datetime import datetime, timedelta

#SYSTEM_PROMPT = ""

#user_prompt = "Give me 5 non-formal ways to say 'See you later' in French."

#messages = [
    #{
        #"role": "system",
        #"content": SYSTEM_PROMPT
    #},
    #{
        #"role": "user",
        #"content": user_prompt
    #},
#]

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
llm = LLM(model=model_name, tokenizer_mode="mistral")

sampling_params = SamplingParams(max_tokens=60, temperature=0.15)
outputs = llm.chat(messages, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)
# Here are five non-formal ways to say "See you later" in French:

# 1. **À plus tard** - Until later
# 2. **À toute** - See you soon (informal)
# 3. **Salut** - Bye (can also mean hi)
# 4. **À plus** - See you later (informal)
# 5. **Ciao** - Bye (informal, borrowed from Italian)

# ```
#  /\_/\
# ( o.o )
#  > ^ <
# ```


ModuleNotFoundError: No module named 'vllm'

### Llama-3.1-8B-Instruct

In [58]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token # we need to specify the pad token specifically
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

In [37]:
llama_prompt = "Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. Det er MEGET vigtigt, at du kun skal gøre dette med maks 50 tokens. Du skal heller ikke bare give mig et enkelt ord som svar. Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."

In [7]:
new_prompt = "Du er deltager i et panel, der diskuterer personlige dilemmaer. Giv mig ikke andet end svaret på dilemmaet. Du må ikke kun give et enkelt ord som svar. Det er meget vigtigt, at du kun bruger maks 50 tokens på svaret."

In [50]:
gpt_prompt = """OPGAVE:
Returnér KUN et råd.

FORMAT (SKAL OVERHOLDES):
- Præcis én sammenhængende tekstblok.
- Ingen indledning, ingen afslutning.
- Ingen linjeskift.
- Ingen anførselstegn.

SPROGREGLER (ABSOLUTTE):
- INGEN personlige pronomener (fx JEG, DU, DIG, mig, MIN, MIT, mine).
- Ingen personlige oplevelser eller fortællinger.
- Neutral, upersonlig tone.

INDHOLD:
- Rådet skal være konkret og handlingsrettet.
- Må ikke bestå af kun ét ord.

LÆNGDE:
- Maksimalt 80 tokens. Stop automatisk ved 80 tokens.

FEJL:
- Hvis nogen regel brydes, er svaret ugyldigt og skal afkortes.

EKSEMPLER PÅ RÅD:

- Træde varsomt i kommunikationen med den allernærmeste familie: tage emnet op forsigtigt og vælge sine ord, fordi man skal være påpasselig i nære familiære relationer.
- Appellere til klimamæssige og miljømæssige hensyn over for svigerfamilien og bede om brugt legetøj (fx fra genbrugsbutikker) som alternativ til nyt, billigt legetøj

"""



In [60]:
meta = "Du er en hjælpsom assistent."

meta_prompt = f"Jeg har givet denne prompt til dig, men du kan ikke finde ud af at følge den. Hvad skal jeg ændre på, så du overholder retningslinjerne? Prompten: {gpt_prompt}. hvad skal jeg gøre?"

In [61]:
#messages = set_prompts(llama_prompt, df['first_person_dilemma'].iloc[1000])
messages = set_prompts(meta, meta_prompt)

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

print(messages)
outputs = model.generate(
	**inputs, 
	max_new_tokens=1000,
	temperature=0.3,
	do_sample=True
	)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': [{'type': 'text', 'text': 'Du er en hjælpsom assistent.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Her er dilemmaet: Jeg har givet denne prompt til dig, men du kan ikke finde ud af at følge den. Hvad skal jeg ændre på, så du overholder retningslinjerne? Prompten: OPGAVE:\nReturnér KUN et råd.\n\nFORMAT (SKAL OVERHOLDES):\n- Præcis én sammenhængende tekstblok.\n- Ingen indledning, ingen afslutning.\n- Ingen linjeskift.\n- Ingen anførselstegn.\n\nSPROGREGLER (ABSOLUTTE):\n- INGEN personlige pronomener (fx JEG, DU, DIG, mig, MIN, MIT, mine).\n- Ingen personlige oplevelser eller fortællinger.\n- Neutral, upersonlig tone.\n\nINDHOLD:\n- Rådet skal være konkret og handlingsrettet.\n- Må ikke bestå af kun ét ord.\n\nLÆNGDE:\n- Maksimalt 80 tokens. Stop automatisk ved 80 tokens.\n\nFEJL:\n- Hvis nogen regel brydes, er svaret ugyldigt og skal afkortes.\n\nEKSEMPLER PÅ RÅD:\n\n- Træde varsomt i kommunikationen med den allernærmeste familie: tage em

In [ ]:
df['first_person_dilemma'].iloc[600]

'Vi skal invitere til børnefødselsdag, men i min svigerfamilie er der en tendens til altid at købe fødselsdagsgaver til vores to små børn — og også til hinandens børn — i Nettos spotvareafdeling. Jeg er simpelthen så træt af at modtage legetøj af så ringe kvalitet, at det går i stykker kort efter. Hjemme går vi generelt op i at købe kvalitet til børnene — både legetøj, tøj og møbler — og vi er alle enige om et budget på 100–150 kroner til hinandens børn, så man sagtens kan finde noget i andre butikker. Nu hvor vi skal holde børnefødselsdag: kan jeg tillade mig at bede gæsterne om ikke at give gaver fra Netto, når min mand mener, at jeg ikke kan?'

In [34]:
df['acceptable_solutions'].iloc[600]

["Sæt et skilt på hoveddøren (fx 'Jeg lufter bare ud' eller 'Do not disturb') så naboer, venner og bekendte forstår, at døren står åben pga. udluftning og ikke er en invitation til at gå ind.",
 "Sig høfligt fra når nogen stikker hovedet ind: 'Det er sødt af dig, men jeg er lige i gang med noget andet – en anden dag.' (gælder overfor venner, bekendte og naboer) — bemærk at det kræver social energi at give afvisningen.",
 'Sæt en fysisk forhindring foran døren (fx en stol eller noget haveaffald) for at signalere, at det ikke er velkomment at træde ind (men vær opmærksom på at folk måske sætter sig og vil snakke).',
 'Luk hoveddøren og åbn andre vinduer/døre i lejligheden for at skabe gennemtræk uden at invitere folk ind gennem hoveddøren.',
 'Stå i døren og stir på dem uden at sige noget, så de forstår, at hun ikke ønsker besøg (en passiv måde at gøre opmærksom på, at de ikke skal gå ind).',
 'Opsæt en rød snor med guldpinde (som en VIP-afspærring) foran døren som en tydelig/legende mar

### Testing batch processing + batch prompting

In [42]:
prompts = []
for i, row in df.iterrows():
    prompt = set_prompts(row['first_person_dilemma'])
    prompts.append(prompt)

df['prompts'] = prompts

In [43]:
# convert df to huggingface dataset

ds = datasets.Dataset.from_pandas(df)

In [55]:
import datasets
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

# specify tokenizer & model

In [56]:
ds_subset = ds.select(range(20))

In [48]:
ds_subset['prompts']

Column([[{'content': [{'text': "Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. Det er MEGET vigtigt, at du kun skal gøre dette med maks 50 tokens. Du skal heller ikke bare give mig et enkelt ord som svar. Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet. Du skal ikke adressere personen, rådet er til.", 'type': 'text'}], 'role': 'system'}, {'content': [{'text': "Her er dilemmaet: Mit navn er Mette, og vores veninde Amalie skal giftes til foråret, så hun skal selvfølgelig have en polterabend. Jeg taler på vegne af alle os, der skal deltage, for vi er havnet i et dilemma. Amalie har givet udtryk over for sin mor, at hun rigtig gerne vil vide, hvornår polterabend'en finder sted. Hun har ondt i maven over, at hun ikke kender datoen, og hun har desuden sagt til sin kommende mand, at hun bliver vr

In [ ]:
messages = ds_subset[0]["prompts"]

prompt_text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False,
)
print(prompt_text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

[{'text': "Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. Det er MEGET vigtigt, at du kun skal gøre dette med maks 50 tokens. Du skal heller ikke bare give mig et enkelt ord som svar. Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet. Du skal ikke adressere personen, rådet er til.", 'type': 'text'}]<|eot_id|><|start_header_id|>user<|end_header_id|>

[{'text': "Her er dilemmaet: Mit navn er Mette, og vores veninde Amalie skal giftes til foråret, så hun skal selvfølgelig have en polterabend. Jeg taler på vegne af alle os, der skal deltage, for vi er havnet i et dilemma. Amalie har givet udtryk over for sin mor, at hun rigtig gerne vil vide, hvornår polterabend'en finder s

In [ ]:
def tokenize_chat(batch):
    chat_prompts = [
        # convert prompts into a chat template
        tokenizer.apply_chat_template( # create chat template used by the loaded model
            messages,
            add_generation_prompt=True,
            tokenize=False, # not tokenizing yet
        )
        for messages in batch["prompts"]
    ]

    # tokenize the chat_template prompts
    tokenized_batch = tokenizer(
        chat_prompts,
        padding=True, # need to add padding for gpu compatability; add padding if sentences do not have the same amount of tokens
        truncation=True,
        return_tensors=None,
    )

    return tokenized_batch

In [ ]:
tokenized_ds = ds_subset.map(
    tokenize_chat,
    batched=True,
    remove_columns=ds_subset.column_names,
)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [41]:
!pip install accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 5.4 MB/s eta 0:00:00a 0:00:01


### Test Qwen-8B

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto"
    #device_map="auto"
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [46]:
def set_prompts_qwen(system_prompt, dilemma):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Her er dilemmaet: {dilemma}. Råd: "}
    ]


In [49]:
llama_prompt

"Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. Det er MEGET vigtigt, at du kun skal gøre dette med maks 50 tokens. Du skal heller ikke bare give mig et enkelt ord som svar. Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."

In [53]:
messages = set_prompts_qwen(llama_prompt, df['first_person_dilemma'].iloc[1000])

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=80
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
#try:
    # rindex finding 151668 (</think>)
 #   index = len(output_ids) - output_ids[::-1].index(151668)
#except ValueError:
 #   index = 0

#thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[0:], skip_special_tokens=True).strip("\n")

#print("thinking content:", thinking_content)
print(content)

Betale, men sige klart, at du ikke kan fortsætte med at betale, når det fordobles.


In [56]:
df['first_person_dilemma'].iloc[1000]

'Jeg skiftede for omtrent et år siden fra det private til det offentlige. I min nye afdeling har man længe haft en morgenmadsordning, hvor man på skift står for at købe ind til fælles morgenmad hver anden uge. Da der er mange i afdelingen, bliver det til mange omgange på et år, og det ender som minimum omkring 1.200 kr. årligt for mig. Derudover er der en gavekasse, som samlet set medfører en årlig udgift på omkring 2.000 kr. Jeg bakker gerne op om det sociale på arbejdspladsen og er en del af en gruppe, som frivilligt arrangerer sociale aktiviteter efter arbejdstid, men jeg mener principielt, at det ikke skal koste så mange penge at være en del af en arbejdsplads. Jeg synes, det er dårligt, at der ikke er afsat penge i afdelingsbudgettet til en månedlig morgenmad, og jeg mener også, at arbejdspladsen burde stå for at sende blomster ved barsel og ved fratrædelser. Dilemmaet er blevet mere aktuelt, fordi der er fremsat forslag om at have morgenmad hver uge, hvilket vil fordoble udgifter

In [57]:
df['acceptable_solutions'].iloc[1000]

["Stå frem offentligt og sig fra (starte en lille 'revolution') — for eksempel rejse dig op og markere over for kollegerne; gør det charmerende og humoristisk (fx Spartacus-parodi) så du ikke fremstår som pedantisk; involver: Linea og de kolleger, der vil bakke op; betingelse: gør det på en måde, der får andre med.",
 'Medbring din egen morgenmad og spis sammen med kollegerne — deltag socialt uden at betale til morgenmadsordningen; involver: Linea og kollegerne ved fælles morgenmad.',
 'Meld dig ud af ordningerne / stop med at betale eller deltage, hvis det strider imod dine principper — vælg at stå udenfor de frivillige ordninger; involver: Linea (konsekvens: mulig sladder/stigmatisering, men legitim mulighed).',
 'Fortsæt med at betale for at bevare den gode stemning — vælg at bidrage økonomisk hvis du foretrækker at undgå konflikt og holde ro i afdelingen; involver: Linea.',
 'Brug dokumentation (fx et Excel-ark) og tal konkret om økonomien — vis hvor meget du reelt har brugt om åre